Importing data

In [3]:
movie= pd.read_csv('/kaggle/input/recommendation-system/ml-25m/movies.csv')
ratings = pd.read_csv('/kaggle/input/recommendation-system/ml-25m/ratings.csv')

In [4]:
movie.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


find the movie titles which has length less than 8

In [5]:
l=[]
for i in movie.title:
  if (len(i)<8):
    l.append(i)
l

['Honky',
 'Viva',
 'Ittefaq',
 '101次求婚',
 'Vrijdag',
 'Hundra',
 'Veljet',
 'Kshanam',
 '6A',
 'Fant',
 'Freedom',
 'Gleason',
 'Bazodee',
 'MamaBoy',
 'The OA',
 'Cosmos',
 'Genesis',
 'Le Ride',
 'Elián',
 'Aprel',
 'Subdue',
 'Slumber',
 'Kickoff',
 'Trauma',
 'Frangi',
 'Harvest',
 'Heroine',
 'Seat 25',
 'Zero',
 'Tau',
 'Mara',
 'Lottery',
 'Since',
 'Wall',
 'Девочки',
 'BÚÉK',
 'Old Man',
 'Polina',
 'Wicked',
 'cursor',
 'Eli',
 'Burdock',
 'Twice',
 'Luz']

In [6]:
movie.dtypes

movieId     int64
title      object
genres     object
dtype: object

* removing year from title
* removing the year from title only if the length of the title is greater than 7
* if length is less than 7 means it doesn't have title attached with its name

In [7]:
import re
def change_title(title):
  if(len(title) >7):
    title= title[:-7]
  return title
movie["title"] = movie["title"].apply(change_title)
movie.title

0                          Toy Story
1                            Jumanji
2                   Grumpier Old Men
3                  Waiting to Exhale
4        Father of the Bride Part II
                    ...             
62418                             We
62419             Window of the Soul
62420                      Bad Poems
62421                   A Girl Thing
62422        Women of Devil's Island
Name: title, Length: 62423, dtype: object

In [8]:
def new_title(title):
    title = re.sub("[^a-zA-Z0-9 ]", "", title)
    return title
movie["new_title"] = movie["title"].apply(new_title)
movie["new_title"] = movie["new_title"].str.lower()

In [9]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [10]:
movie.isnull().sum()

movieId      0
title        0
genres       0
new_title    0
dtype: int64

In [11]:
ratings.isnull().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

Merging Datasets

In [12]:
movies= movie.merge(ratings, on='movieId')
movies.head()

,movieId,title,genres,new_title,userId,rating,timestamp
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,toy story,2,3.5,1141415820
1,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,toy story,3,4.0,1439472215
2,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,toy story,4,3.0,1573944252
3,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,toy story,5,4.0,858625949
4,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,toy story,8,4.0,890492517


# Creating tables based on popularity

* grouping based on number of ratings given

In [13]:
num_rating= movies.groupby('title').count()['rating'].reset_index()
num_rating.head()

,title,rating
0,"""BLOW THE NIGHT!"" Let's Spend the Night Together",1
1,"""Great Performances"" Cats",179
2,#1 Cheerleader Camp,9
3,#Captured,2
4,#Female Pleasure,3


rename the rating column name to num_of_rating

In [14]:
num_rating.rename(columns = {'rating':'num_of_rating'}, inplace = True)
num_rating.head()

,title,num_of_rating
0,"""BLOW THE NIGHT!"" Let's Spend the Night Together",1
1,"""Great Performances"" Cats",179
2,#1 Cheerleader Camp,9
3,#Captured,2
4,#Female Pleasure,3


grouping based on average number of ratings given

In [15]:
avg_rating= movies.groupby('title').mean()['rating'].reset_index()
avg_rating.head()

,title,rating
0,"""BLOW THE NIGHT!"" Let's Spend the Night Together",3.000000
1,"""Great Performances"" Cats",2.896648
2,#1 Cheerleader Camp,2.111111
3,#Captured,3.750000
4,#Female Pleasure,3.666667


rename the rating column name to num_of_rating

In [16]:
avg_rating.rename(columns = {'rating':'avg_rating'}, inplace = True)
avg_rating

,title,avg_rating
0,"""BLOW THE NIGHT!"" Let's Spend the Night Together",3.000000
1,"""Great Performances"" Cats",2.896648
2,#1 Cheerleader Camp,2.111111
3,#Captured,3.750000
4,#Female Pleasure,3.666667
...,...,...
55471,…And the Fifth Horseman Is Fear,3.250000
55472,キサラギ,3.250000
55473,チェブラーシカ,3.166667
55474,貞子3D,2.375000


merging two data frames


In [17]:
popularity = num_rating.merge(avg_rating,on='title')
popularity.head()


,title,num_of_rating,avg_rating
0,"""BLOW THE NIGHT!"" Let's Spend the Night Together",1,3.000000
1,"""Great Performances"" Cats",179,2.896648
2,#1 Cheerleader Camp,9,2.111111
3,#Captured,2,3.750000
4,#Female Pleasure,3,3.666667


* taking the titles where users have given number of ratings > 1000 
* sorting the titles in descending based on average rating

In [18]:
popularity = popularity[popularity.num_of_rating > 1000].sort_values('avg_rating',ascending=False)
popularity.head(50)

,title,num_of_rating,avg_rating
34644,Planet Earth II,1124,4.483096
34643,Planet Earth,1747,4.464797
39314,"Shawshank Redemption, The",81482,4.413576
4711,Band of Brothers,1356,4.398599
18168,"Godfather, The",52498,4.324336
51973,"Usual Suspects, The",55366,4.284353
18169,"Godfather: Part II, The",34188,4.261759
38998,Seven Samurai (Shichinin no samurai),13367,4.254769
38399,Schindler's List,60411,4.247579
36354,Rear Window,20170,4.237432


# Collaborative based filtering

* group by userids who has given ratings more than 1500 movies

In [19]:

x= movies.groupby('userId').count()['rating']>1500
x.head()

userId
1    False
2    False
3    False
4    False
5    False
Name: rating, dtype: bool

* getting their index of users who has given ratings for more than 1000 movies

In [20]:
sim_users= x[x].index 
sim_users

Int64Index([   548,    626,    847,    997,   1401,   1652,   1748,   1920,
              1977,   2165,
            ...
            160922, 160951, 161184, 161383, 161544, 161586, 161928, 162047,
            162271, 162516],
           dtype='int64', name='userId', length=910)

* all movies that sim_users has also viewed

In [21]:
sim_movies= movies[movies['userId'].isin(sim_users)]
sim_movies

,movieId,title,genres,new_title,userId,rating,timestamp
172,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,toy story,548,4.5,1431644949
199,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,toy story,626,4.5,1136304145
283,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,toy story,847,4.0,1048092664
339,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,toy story,997,4.5,1529250285
477,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,toy story,1401,4.5,1544250123
...,...,...,...,...,...,...,...
25000061,209067,Sousse: Marché aux charbons (avec chameaux),(no genres listed),sousse march aux charbons avec chameaux,154484,2.5,1574026518
25000068,209089,An Impossible Balancing Feat,(no genres listed),an impossible balancing feat,154484,3.0,1574075308
25000070,209103,Tsar Ivan the Terrible,(no genres listed),tsar ivan the terrible,13737,4.0,1574112239
25000079,209135,Jane B. by Agnès V.,Documentary|Fantasy,jane b by agns v,154484,3.5,1574187267


* groupby title for movies the number of ratings given to the movie is greater than 500

In [22]:
y= sim_movies.groupby('title').count()['rating']>500
famous_movies= y[y].index 
famous_movies

Index(['(500) Days of Summer', '10 Things I Hate About You',
       '101 Dalmatians (One Hundred and One Dalmatians)', '12 Angry Men',
       '2001: A Space Odyssey', '21 Grams', '28 Days Later', '300',
       '3:10 to Yuma', '40-Year-Old Virgin, The',
       ...
       'X-Men', 'X-Men: First Class', 'X-Men: The Last Stand',
       'X2: X-Men United', 'You've Got Mail', 'Young Frankenstein', 'Zodiac',
       'Zombieland', 'Zoolander', 'xXx'],
      dtype='object', name='title', length=791)

* getting the details of the movies where the number of ratings given to the movie is greater than 500 

In [23]:

final_ratings= sim_movies[movies['title'].isin(famous_movies)]
final_ratings

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,movieId,title,genres,new_title,userId,rating,timestamp
172,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,toy story,548,4.5,1431644949
199,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,toy story,626,4.5,1136304145
283,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,toy story,847,4.0,1048092664
339,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,toy story,997,4.5,1529250285
477,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,toy story,1401,4.5,1544250123
...,...,...,...,...,...,...,...
24997994,205873,Cinderella,Animation|Children|Fantasy,cinderella,72315,3.0,1567666207
24998980,206899,Charlie's Angels,Action|Adventure|Comedy,charlies angels,132358,2.0,1573872003
24998982,206899,Charlie's Angels,Action|Adventure|Comedy,charlies angels,143568,3.5,1574300863
24998984,206899,Charlie's Angels,Action|Adventure|Comedy,charlies angels,146708,3.0,1574282669


In [24]:
final_ratings.drop_duplicates()

,movieId,title,genres,new_title,userId,rating,timestamp
172,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,toy story,548,4.5,1431644949
199,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,toy story,626,4.5,1136304145
283,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,toy story,847,4.0,1048092664
339,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,toy story,997,4.5,1529250285
477,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,toy story,1401,4.5,1544250123
...,...,...,...,...,...,...,...
24997994,205873,Cinderella,Animation|Children|Fantasy,cinderella,72315,3.0,1567666207
24998980,206899,Charlie's Angels,Action|Adventure|Comedy,charlies angels,132358,2.0,1573872003
24998982,206899,Charlie's Angels,Action|Adventure|Comedy,charlies angels,143568,3.5,1574300863
24998984,206899,Charlie's Angels,Action|Adventure|Comedy,charlies angels,146708,3.0,1574282669



Creating a pivot table

In [25]:
pt= final_ratings.pivot_table(index='new_title',columns='userId',values='rating')
pt1= final_ratings.pivot_table(index=['new_title','title'],columns='userId',values='rating')
pt

userId,548,626,847,997,1401,1652,1748,1920,1977,2165,...,160922,160951,161184,161383,161544,161586,161928,162047,162271,162516
new_title,,,,,,,,,,,,,,,,,,,,,
10 things i hate about you,4.5,4.0,3.5,3.0,1.0,NaN,1.5,NaN,NaN,4.0,...,4.0,3.0,2.5,NaN,NaN,NaN,NaN,3.0,NaN,3.5
101 dalmatians one hundred and one dalmatians,4.0,4.0,3.5,3.0,NaN,5.0,3.0,3.0,4.0,NaN,...,4.0,NaN,4.0,3.5,NaN,2.5,3.0,NaN,NaN,4.0
12 angry men,NaN,4.0,4.0,4.0,5.0,4.5,NaN,NaN,4.0,5.0,...,4.0,NaN,5.0,NaN,NaN,3.5,NaN,5.0,3.5,4.5
2001 a space odyssey,NaN,4.0,4.5,5.0,3.0,3.0,4.0,4.0,4.5,0.5,...,3.0,4.5,5.0,3.0,4.0,5.0,NaN,4.0,4.5,4.5
21 grams,4.0,NaN,2.5,4.0,NaN,NaN,3.5,4.0,3.0,5.0,...,NaN,4.0,4.5,NaN,3.0,NaN,NaN,4.0,2.5,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
young frankenstein,NaN,4.0,3.0,NaN,2.0,NaN,1.5,4.0,NaN,NaN,...,4.5,4.5,4.5,3.0,NaN,4.5,NaN,2.0,3.5,4.0
youve got mail,3.5,4.0,3.5,3.0,NaN,4.0,4.0,3.0,2.0,NaN,...,3.5,3.0,3.0,NaN,1.5,NaN,3.0,2.5,1.5,2.5
zodiac,4.5,NaN,4.0,4.0,4.0,5.0,3.5,3.0,NaN,1.5,...,3.0,4.0,3.0,NaN,2.5,2.0,NaN,5.0,4.0,4.0


In [26]:
pt.fillna(0,inplace=True)
pt1.fillna(0,inplace=True)
pt

userId,548,626,847,997,1401,1652,1748,1920,1977,2165,...,160922,160951,161184,161383,161544,161586,161928,162047,162271,162516
new_title,,,,,,,,,,,,,,,,,,,,,
10 things i hate about you,4.5,4.0,3.5,3.0,1.0,0.0,1.5,0.0,0.0,4.0,...,4.0,3.0,2.5,0.0,0.0,0.0,0.0,3.0,0.0,3.5
101 dalmatians one hundred and one dalmatians,4.0,4.0,3.5,3.0,0.0,5.0,3.0,3.0,4.0,0.0,...,4.0,0.0,4.0,3.5,0.0,2.5,3.0,0.0,0.0,4.0
12 angry men,0.0,4.0,4.0,4.0,5.0,4.5,0.0,0.0,4.0,5.0,...,4.0,0.0,5.0,0.0,0.0,3.5,0.0,5.0,3.5,4.5
2001 a space odyssey,0.0,4.0,4.5,5.0,3.0,3.0,4.0,4.0,4.5,0.5,...,3.0,4.5,5.0,3.0,4.0,5.0,0.0,4.0,4.5,4.5
21 grams,4.0,0.0,2.5,4.0,0.0,0.0,3.5,4.0,3.0,5.0,...,0.0,4.0,4.5,0.0,3.0,0.0,0.0,4.0,2.5,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
young frankenstein,0.0,4.0,3.0,0.0,2.0,0.0,1.5,4.0,0.0,0.0,...,4.5,4.5,4.5,3.0,0.0,4.5,0.0,2.0,3.5,4.0
youve got mail,3.5,4.0,3.5,3.0,0.0,4.0,4.0,3.0,2.0,0.0,...,3.5,3.0,3.0,0.0,1.5,0.0,3.0,2.5,1.5,2.5
zodiac,4.5,0.0,4.0,4.0,4.0,5.0,3.5,3.0,0.0,1.5,...,3.0,4.0,3.0,0.0,2.5,2.0,0.0,5.0,4.0,4.0


cosine simularity

In [27]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_scores = cosine_similarity(pt)
similarity_scores.shape

(791, 791)

In [28]:
similarity_scores

array([[1.        , 0.59671396, 0.58775479, ..., 0.62510637, 0.61193687,
        0.67061426],
       [0.59671396, 1.        , 0.63828935, ..., 0.52034784, 0.52022317,
        0.60651434],
       [0.58775479, 0.63828935, 1.        , ..., 0.69453301, 0.61961764,
        0.64625129],
       ...,
       [0.62510637, 0.52034784, 0.69453301, ..., 1.        , 0.74656204,
        0.65302658],
       [0.61193687, 0.52022317, 0.61961764, ..., 0.74656204, 1.        ,
        0.68600489],
       [0.67061426, 0.60651434, 0.64625129, ..., 0.65302658, 0.68600489,
        1.        ]])

In [29]:
def recommend_movie(title):
  title= title.lower()
  title= new_title(title)
  print(title)
  index= np.where(pt.index==title)[0][0]
  print(index)
  sim_scores= enumerate(similarity_scores[index])
  sim_scores= sorted(sim_scores, key= lambda x: x[1], reverse=True)
  sim_scores= sim_scores[1:11]
  sim_index= [i[0] for i in sim_scores]
  for i in sim_index:
    print(pt1.index[i][1])

In [30]:
recommend_movie('12 Angry Men')

One Flew Over the Cuckoo's Nest
Rear Window
Godfather, The
Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb
Casablanca
Fargo
Goodfellas
Taxi Driver
Pulp Fiction
2001: A Space Odyssey


In [31]:
recommend_movie('toy story')

Toy Story 2
Jurassic Park
Back to the Future
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark)
Star Wars: Episode IV - A New Hope
Sixth Sense, The
Monsters, Inc.
Matrix, The
Silence of the Lambs, The
Star Wars: Episode V - The Empire Strikes Back
